In [ ]:
!pip install earthengine-api
!pip install earthengine-api pandas
!pip install geopandas
!pip install gdal
!pip install geemap
!pip install mss

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolut

In [ ]:
import pandas as pd

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv('GLC03122015_20241006.csv', sep=',', encoding='utf-8')

# Loại bỏ khoảng trắng ở đầu và cuối tên cột
df.columns = df.columns.str.strip()

# Chuyển đổi cột 'date_' sang định dạng datetime và loại bỏ múi giờ
df['date_'] = pd.to_datetime(df['date_'], errors='coerce').dt.tz_convert(None)

# Lọc dữ liệu cho các sự kiện xảy ra ở Việt Nam
vietnam_landslides = df[df['country'] == 'Vietnam']



In [ ]:
# Xác thực tài khoản Earth Engine
import ee
ee.Authenticate()

In [ ]:
import ee
import pandas as pd

# Khởi tạo Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')

# Tạo danh sách chứa kết quả lượng mưa và độ ẩm đất
data = []

# Lặp qua từng bản ghi trong DataFrame
for index, row in vietnam_landslides.iterrows():
    date = row['date_']  # Ngày
    latitude = row['latitude']  # Vĩ độ
    longitude = row['longitude']  # Kinh độ

    # Định nghĩa ngày bắt đầu và kết thúc
    start_date = (date - pd.Timedelta(days=4)).strftime('%Y-%m-%d')
    end_date = (date + pd.Timedelta(days=0)).strftime('%Y-%m-%d')

    # Lấy dữ liệu lượng mưa từ CHIRPS
    precipitation_image = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
                           .filterDate(start_date, end_date)
                           .mean()
                           .reduceRegion(
                               reducer=ee.Reducer.mean(),
                               geometry=ee.Geometry.Point(longitude, latitude),
                               scale=5000
                           ))

    rainfall = precipitation_image.get('precipitation').getInfo()

    # Lấy dữ liệu độ ẩm đất từ ERA5
    soil_moisture_image = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
                           .filterDate(start_date, end_date)
                           .mean()
                           .reduceRegion(
                               reducer=ee.Reducer.mean(),
                               geometry=ee.Geometry.Point(longitude, latitude),
                               scale=1000
                           ))

    soil_moisture = soil_moisture_image.get('skin_reservoir_content').getInfo() if soil_moisture_image else None


    # Thêm dữ liệu vào danh sách
    data.append({
        'date': date.date(),
        'latitude': latitude,
        'longitude': longitude,
        'precipitation': rainfall,
        'soil_moisture': soil_moisture
    })

# Chuyển danh sách thành DataFrame
result_df = pd.DataFrame(data)

# In kết quả
result_df


,date,latitude,longitude,precipitation,soil_moisture
0,2012-09-27,12.1639,107.4326,10.529050,0.000627
1,2010-08-17,21.3161,107.3397,24.109480,0.000364
2,2009-09-29,14.8733,108.0191,56.060215,0.000637
3,2012-04-02,12.2736,108.9163,30.150015,0.000884
4,2012-09-09,21.8777,104.5658,19.571638,0.000539
...,...,...,...,...,...
91,2007-07-01,21.0233,107.2930,10.957873,NaN
92,2011-04-01,18.9954,105.4625,3.539586,0.000185
93,2007-11-09,15.3333,108.2167,0.000000,0.000185
94,2009-11-05,15.5721,108.4671,45.234474,0.000367


In [ ]:
# Lọc ra các hàng có giá trị precipitation bằng 0
zero_rainfall_df = rainfall_df[rainfall_df['precipitation'] == 0]

# In kết quả
zero_rainfall_df


,date,latitude,longitude,precipitation
10,2013-01-31,10.0222,106.5408,0.0
19,2011-09-13,22.3405,103.9989,0.0
25,2013-11-22,14.9627,108.3574,0.0
40,2013-08-25,12.0467,108.4403,0.0
58,2010-04-25,22.7662,104.9389,0.0
79,2008-01-04,20.5667,105.9000,0.0
82,2011-11-25,14.7430,108.7301,0.0
85,2011-11-26,15.1257,108.8115,0.0
95,2008-09-25,23.1667,105.4167,0.0


In [ ]:
# Làm tròn giá trị precipitation xuống 3 chữ số thập phân
rainfall_df['precipitation'] = rainfall_df['precipitation'].round(3)

In [ ]:
rainfall_df

,date,latitude,longitude,precipitation
0,2012-09-27,12.1639,107.4326,119.839
1,2010-08-17,21.3161,107.3397,136.782
2,2009-09-29,14.8733,108.0191,299.961
3,2012-04-02,12.2736,108.9163,120.600
4,2012-09-09,21.8777,104.5658,121.976
...,...,...,...,...
91,2007-07-01,21.0233,107.2930,190.115
92,2011-04-01,18.9954,105.4625,14.158
93,2007-11-09,15.3333,108.2167,333.756
94,2009-11-05,15.5721,108.4671,300.318


In [ ]:
rainfall_df.describe()

,latitude,longitude,precipitation
count,96.000000,96.000000,96.000000
mean,17.807793,106.550532,198.230562
std,3.981802,1.891827,157.180442
min,8.831300,102.620000,0.000000
25%,15.026275,104.915000,76.884500
50%,16.461950,106.715000,145.818500
75%,21.709275,108.356650,303.387000
max,23.229100,109.216400,625.735000


In [ ]:
# Định nghĩa ngưỡng lượng mưa
threshold = 625

# Lọc ra các hàng có lượng mưa lớn hơn ngưỡng
high_rainfall_df = rainfall_df[rainfall_df['precipitation'] > threshold]

# In kết quả
print(high_rainfall_df)


          date  latitude  longitude  precipitation
86  2008-11-20   15.1238   108.8117        625.735


In [ ]:
import ee
import pandas as pd

# Khởi tạo Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')
# Chọn khoảng thời gian (có thể thay đổi theo nhu cầu)
start_date = '2000-01-01'
end_date = '2023-12-31'

# Tải bộ dữ liệu về sự kiện lũ
flood_events = ee.FeatureCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1") \
                .filterDate(start_date, end_date)

# Hàm trích xuất thông tin
def extract_info(feature):
    return ee.Feature(None, {
        'id': feature.get('id'),
        'cc': feature.get('cc'),
        'countries': feature.get('countries'),
        'dfo_centroid_x': feature.get('dfo_centroid_x'),
        'dfo_centroid_y': feature.get('dfo_centroid_y'),
        'dfo_country': feature.get('dfo_country'),
        'dfo_other_country': feature.get('dfo_other_country'),
        'dfo_displaced': feature.get('dfo_displaced'),
        'dfo_main_cause': feature.get('dfo_main_cause'),
        'dfo_severity': feature.get('dfo_severity'),
        'dfo_dead': feature.get('dfo_dead'),
        'dfo_validation_type': feature.get('dfo_validation_type'),
        'glide_index': feature.get('glide_index'),
        'gfd_country_code': feature.get('gfd_country_code'),
        'gfd_country_name': feature.get('gfd_country_name'),
        'composite_type': feature.get('composite_type'),
        'threshold_type': feature.get('threshold_type'),
        'threshold_b1b2': feature.get('threshold_b1b2'),
        'threshold_b7': feature.get('threshold_b7'),
        'otsu_sample_res': feature.get('otsu_sample_res'),
        'slope_threshold': feature.get('slope_threshold')
    })

# Áp dụng hàm để lấy thông tin
features = flood_events.map(extract_info)

# Lấy dữ liệu và lưu vào DataFrame
try:
    result = features.getInfo()  # Lấy dữ liệu
    data = []  # Danh sách để lưu trữ dữ liệu

    # Lưu vào danh sách
    for feature in result['features']:
        properties = feature['properties']
        data.append({
            'id': properties['id'],
            'cc': properties['cc'],
            'countries': properties['countries'],
            'dfo_centroid_x': properties['dfo_centroid_x'],
            'dfo_centroid_y': properties['dfo_centroid_y'],
            'dfo_country': properties['dfo_country'],
            'dfo_other_country': properties['dfo_other_country'],
            'dfo_displaced': properties['dfo_displaced'],
            'dfo_main_cause': properties['dfo_main_cause'],
            'dfo_severity': properties['dfo_severity'],
            'dfo_dead': properties['dfo_dead'],
            'dfo_validation_type': properties['dfo_validation_type'],
            'glide_index': properties['glide_index'],
            'gfd_country_code': properties['gfd_country_code'],
            'gfd_country_name': properties['gfd_country_name'],
            'composite_type': properties['composite_type'],
            'threshold_type': properties['threshold_type'],
            'threshold_b1b2': properties['threshold_b1b2'],
            'threshold_b7': properties['threshold_b7'],
            'otsu_sample_res': properties['otsu_sample_res'],
            'slope_threshold': properties['slope_threshold']
        })

    # Chuyển danh sách thành DataFrame
    df = pd.DataFrame(data)

    # Hiển thị DataFrame
    df.head()  # Hiển thị 5 dòng đầu tiên

except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
df.columns

Index(['id', 'cc', 'countries', 'dfo_centroid_x', 'dfo_centroid_y',
       'dfo_country', 'dfo_other_country', 'dfo_displaced', 'dfo_main_cause',
       'dfo_severity', 'dfo_dead', 'dfo_validation_type', 'glide_index',
       'gfd_country_code', 'gfd_country_name', 'composite_type',
       'threshold_type', 'threshold_b1b2', 'threshold_b7', 'otsu_sample_res',
       'slope_threshold'],
      dtype='object')

In [ ]:
df

,id,cc,countries,dfo_centroid_x,dfo_centroid_y,dfo_country,dfo_other_country,dfo_displaced,dfo_main_cause,dfo_severity,...,dfo_validation_type,glide_index,gfd_country_code,gfd_country_name,composite_type,threshold_type,threshold_b1b2,threshold_b7,otsu_sample_res,slope_threshold
0,1586,AUS,Australia,143.697800,-31.268059,Australia,NA,200,Monsoonal rain,2.0,...,News,NA,['AS'],['AUSTRALIA'],3Day,otsu,0.711,1815.18,231.66,5
1,1587,MDG,Madagascar,47.295670,-15.782624,Madagascar,NA,800000,Tropical cyclone,1.0,...,News,NA,['MA'],['MADAGASCAR'],3Day,otsu,1.056,839.16,231.66,5
2,1595,"UKR, POL, ROU, BGR, BLR, SRB, HRV, HUN, BIH, SVN","Ukraine, Poland, Romania, Bulgaria, Belarus, S...",22.415404,46.763746,Romania,Hungary,623,Snowmelt,2.0,...,News,NA,"['IT', 'HR', 'AL', 'BK', 'MJ', 'RB', 'MK', 'KV...","['ITALY', 'CROATIA', 'ALBANIA', 'BOSNIA & HERZ...",3Day,otsu,0.743,1064.36,231.66,5
3,1614,"VNM, THA, LAO, KHM","Vietnam, Thailand, Laos, Cambodia",105.063841,11.242567,Thailand,Vietnam,25000,Monsoonal rain,1.0,...,News,NA,"['TH', 'BM', 'LA', 'CB', 'VM']","['THAILAND', 'BURMA', 'LAOS', 'CAMBODIA', 'VIE...",3Day,otsu,1.108,839.69,231.66,5
4,1627,"RUS, CHN, KOR, PRK","Russia, China, Korea, Republic of, North Korea",132.057679,43.773883,China,Russia,14140,Tropical cyclone,1.0,...,News,NA,"['CH', 'KS', 'UU', 'KN', 'RS']","['CHINA', 'SOUTH KOREA', 'Korean islands under...",3Day,otsu,1.005,1319.71,231.66,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,4683,"GHA, TGO",None,-1.066060,9.537550,Ghana,,100000,"Heavy Rain, Dam releases",1.5,...,FloodList,,"['UV', 'TO', 'GH', 'IV', 'BN', 'UU']","['BURKINA FASO', 'TOGO', 'GHANA', ""COTE D'IVOI...",3Day,standard,0.700,675.00,NaN,5
909,4695,"MEX, USA",None,-105.236000,23.721800,Mexico,,0,Tropical Storm Willa,1.5,...,News Reports,,"['US', 'MX']","['UNITED STATES', 'MEXICO']",3Day,standard,0.700,675.00,NaN,5
910,4703,"BRA, BOL, PRY, ARG",None,-57.085300,-25.342700,Paraguay,,10000,Heavy Rain,1.0,...,FloodList,,"['BR', 'BL', 'PA', 'AR']","['BRAZIL', 'BOLIVIA', 'PARAGUAY', 'ARGENTINA']",3Day,otsu,0.930,600.35,231.66,5
911,4704,"KHM, VNM, LAO",None,107.137000,11.491000,Vietnam,,100000,Tropical Storm Usagi,1.0,...,FloodList,,"['TH', 'LA', 'CB', 'VM']","['THAILAND', 'LAOS', 'CAMBODIA', 'VIETNAM']",3Day,standard,0.700,675.00,NaN,5


Dưới đây là giải thích cho từng cột trong DataFrame mà bạn đã lấy từ bộ dữ liệu sự kiện lũ quét:

1. **id**:
   - Mã danh sách duy nhất của sự kiện lũ. Mỗi sự kiện lũ sẽ có một mã ID khác nhau.

2. **cc**:
   - Mã quốc gia theo chuẩn ISO ba chữ cái. Đây là danh sách các mã cho các quốc gia mà lũ nước đã được phát hiện trong các lưu vực liên quan đến sự kiện lũ.

3. **countries**:
   - Tên các quốc gia (dưới dạng danh sách) mà nước lũ đã được phát hiện trong các lưu vực liên quan đến sự kiện lũ.

4. **dfo_centroid_x**:
   - Kinh độ của tâm điểm (centroid) của hình dạng sự kiện lũ, ước lượng vị trí của sự kiện lũ theo cơ sở dữ liệu của Dartmouth Flood Observatory (DFO).

5. **dfo_centroid_y**:
   - Vĩ độ của tâm điểm (centroid) của hình dạng sự kiện lũ, tương tự như cột kinh độ, nó ước lượng vị trí của sự kiện lũ.

6. **dfo_country**:
   - Quốc gia chính có liên quan đến sự kiện lũ. Đây là quốc gia mà sự kiện lũ ảnh hưởng nhiều nhất.

7. **dfo_other_country**:
   - Quốc gia thứ hai có liên quan đến sự kiện lũ, nếu có. Điều này thể hiện những quốc gia khác mà lũ có thể đã ảnh hưởng.

8. **dfo_displaced**:
   - Số ước tính tổng số người đã phải rời bỏ nhà cửa hoặc bị di dời do sự kiện lũ.

9. **dfo_main_cause**:
   - Nguyên nhân chính gây ra sự kiện lũ. Đây có thể là các yếu tố như mưa lớn, tan chảy băng tuyết, hoặc các nguyên nhân tự nhiên khác.

10. **dfo_severity**:
    - Mức độ nghiêm trọng của sự kiện lũ. Thang điểm có thể như sau:
      - 1: Lũ lớn, gây thiệt hại đáng kể cho cấu trúc hoặc nông nghiệp, có thương vong và/hoặc có khoảng thời gian 5-15 năm từ lần sự kiện lũ tương tự gần nhất.
      - 1.5: Các sự kiện rất lớn: có tần suất lớn hơn 15 năm nhưng nhỏ hơn 100 năm.
      - 2: Các sự kiện cực đoan: có tần suất lớn hơn 100 năm.

11. **dfo_dead**:
    - Số ước tính các ca tử vong do sự kiện lũ. Đây là con số người đã mất mạng do lũ.

12. **dfo_validation_type**:
    - Loại nguồn xác nhận chính cho sự kiện lũ. Điều này có thể là báo cáo từ các cơ quan, thông tin từ truyền thông, hoặc các nguồn khác.

13. **glide_index**:
    - Số nhận diện toàn cầu (Global Identifier Number). Đây là một mã số duy nhất cho mỗi sự kiện lũ.

14. **gfd_country_code**:
    - Danh sách mã quốc gia hai chữ cái theo chuẩn FIPS cho các quốc gia có liên quan đến lưu vực được sử dụng trong thuật toán phát hiện nước.

15. **gfd_country_name**:
    - Tên các quốc gia (dưới dạng danh sách) có liên quan đến lưu vực được sử dụng trong thuật toán phát hiện nước.

16. **composite_type**:
    - Số ngày được sử dụng để tạo ra composite trong thuật toán phát hiện nước.

17. **threshold_type**:
    - Loại ngưỡng được sử dụng để phân loại nước/không nước trong thuật toán phát hiện nước - có thể là "otsu" hoặc "standard".

18. **threshold_b1b2**:
    - Giá trị ngưỡng được áp dụng cho tỷ lệ b2/b1 được sử dụng trong thuật toán phát hiện nước.

19. **threshold_b7**:
    - Giá trị ngưỡng được áp dụng cho băng tần 7 (SWIR) trong thuật toán phát hiện nước.

20. **otsu_sample_res**:
    - Độ phân giải không gian (tính bằng mét) của bộ giảm được sử dụng để xây dựng mosaic MODIS, từ đó mẫu và ước lượng ngưỡng Otsu (chỉ có sẵn cho các sự kiện lũ đã sử dụng ngưỡng Otsu và không phải là ngưỡng mặc định).

21. **slope_threshold**:
    - Giá trị được sử dụng để loại bỏ các khu vực dốc khỏi thuật toán phát hiện nước nhằm giảm thiểu sai số do bóng đổ từ địa hình.

Nếu bạn có bất kỳ câu hỏi nào khác hoặc cần thêm thông tin, hãy cho tôi biết!

In [ ]:
import ee
import pandas as pd


# Định nghĩa bộ sưu tập
flood_events = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")

# Lấy toàn bộ thông tin về sự kiện lũ lụt
flood_list = flood_events.getInfo()

# Lưu thông tin vào danh sách
events_info = []
for feature in flood_list['features']:
    properties = feature['properties']
    event_data = {
        'event_id': properties.get('system:index'),  # ID sự kiện
        'event_start': ee.Date(properties.get('system:time_start')).format('YYYY-MM-dd').getInfo(),  # Ngày bắt đầu
        'event_end': ee.Date(properties.get('system:time_end')).format('YYYY-MM-dd').getInfo(),  # Ngày kết thúc
    }

    # Lưu tất cả các thuộc tính khác (nếu cần)
    for key, value in properties.items():
        event_data[key] = value

    events_info.append(event_data)

df = pd.DataFrame(events_info)


EEException: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

In [ ]:
import pandas as pd


# Danh sách các cột cần giữ lại
columns_to_keep = [
    'event_start',        # Ngày bắt đầu của sự kiện lũ lụt
    'event_end',          # Ngày kết thúc của sự kiện lũ lụt
    'dfo_centroid_x',     # Tọa độ x của trọng tâm (centroid) sự kiện lũ lụt
    'dfo_centroid_y',     # Tọa độ y của trọng tâm (centroid) sự kiện lũ lụt
    'dfo_main_cause',     # Nguyên nhân chính gây ra sự kiện lũ lụt
    'gfd_country_name',    # Tên quốc gia nơi xảy ra sự kiện lũ lụt
    'glide_index',        # Chỉ số GLIDE (Global Disaster Identifier) của sự kiện lũ lụt
    'slope_threshold',     # Ngưỡng độ dốc liên quan đến sự kiện lũ lụt
    'dfo_severity',       # Mức độ nghiêm trọng của sự kiện lũ lụt
    'dfo_displaced',      # Số lượng người bị di dời do sự kiện lũ lụt
    'dfo_validation_type', # Loại xác thực cho sự kiện lũ lụt
    'composite_type',     # Loại tổ hợp của dữ liệu liên quan đến sự kiện
    'dfo_country',        # Quốc gia mà sự kiện lũ lụt ảnh hưởng
    'countries',          # Các quốc gia khác có thể bị ảnh hưởng bởi sự kiện lũ lụt
    'dfo_other_country',  # Tên các quốc gia khác liên quan đến sự kiện
    'dfo_dead',           # Số lượng người thiệt mạng do sự kiện lũ lụt
    'gfd_country_code',    # Mã quốc gia theo tiêu chuẩn GFD (Global Flood Database)
    'threshold_type',     # Loại ngưỡng mà sự kiện lũ lụt được xác định
    'threshold_b7'       # Ngưỡng giá trị cho băng 7 (band 7) liên quan đến lũ lụt
]


df = df[columns_to_keep]
df


,event_start,event_end,dfo_centroid_x,dfo_centroid_y,dfo_main_cause,gfd_country_name,glide_index,slope_threshold,dfo_severity,dfo_displaced,dfo_validation_type,composite_type,dfo_country,countries,dfo_other_country,dfo_dead,gfd_country_code,threshold_type,threshold_b7
0,2000-02-18,2000-03-01,143.697800,-31.268059,Monsoonal rain,['AUSTRALIA'],NA,5,2.0,200,News,3Day,Australia,Australia,NA,1,['AS'],otsu,1815.18
1,2000-02-17,2000-03-11,47.295670,-15.782624,Tropical cyclone,['MADAGASCAR'],NA,5,1.0,800000,News,3Day,Madagascar,Madagascar,NA,200,['MA'],otsu,839.16
2,2000-04-05,2000-04-25,22.415404,46.763746,Snowmelt,"['ITALY', 'CROATIA', 'ALBANIA', 'BOSNIA & HERZ...",NA,5,2.0,623,News,3Day,Romania,"Ukraine, Poland, Romania, Bulgaria, Belarus, S...",Hungary,10,"['IT', 'HR', 'AL', 'BK', 'MJ', 'RB', 'MK', 'KV...",otsu,1064.36
3,2000-07-11,2000-08-10,105.063841,11.242567,Monsoonal rain,"['THAILAND', 'BURMA', 'LAOS', 'CAMBODIA', 'VIE...",NA,5,1.0,25000,News,3Day,Thailand,"Vietnam, Thailand, Laos, Cambodia",Vietnam,33,"['TH', 'BM', 'LA', 'CB', 'VM']",otsu,839.69
4,2000-08-30,2000-09-10,132.057679,43.773883,Tropical cyclone,"['CHINA', 'SOUTH KOREA', 'Korean islands under...",NA,5,1.0,14140,News,3Day,China,"Russia, China, Korea, Republic of, North Korea",Russia,31,"['CH', 'KS', 'UU', 'KN', 'RS']",otsu,1319.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,2018-09-01,2018-10-02,-1.066060,9.537550,"Heavy Rain, Dam releases","['BURKINA FASO', 'TOGO', 'GHANA', ""COTE D'IVOI...",,5,1.5,100000,FloodList,3Day,Ghana,NaN,,34,"['UV', 'TO', 'GH', 'IV', 'BN', 'UU']",standard,675.00
909,2018-10-23,2018-10-27,-105.236000,23.721800,Tropical Storm Willa,"['UNITED STATES', 'MEXICO']",,5,1.5,0,News Reports,3Day,Mexico,NaN,,0,"['US', 'MX']",standard,675.00
910,2018-10-29,2018-11-07,-57.085300,-25.342700,Heavy Rain,"['BRAZIL', 'BOLIVIA', 'PARAGUAY', 'ARGENTINA']",,5,1.0,10000,FloodList,3Day,Paraguay,NaN,,0,"['BR', 'BL', 'PA', 'AR']",otsu,600.35
911,2018-11-24,2018-11-29,107.137000,11.491000,Tropical Storm Usagi,"['THAILAND', 'LAOS', 'CAMBODIA', 'VIETNAM']",,5,1.0,100000,FloodList,3Day,Vietnam,NaN,,1,"['TH', 'LA', 'CB', 'VM']",standard,675.00


In [ ]:
# Lưu DataFrame vào file CSV
df.to_csv('flood_events_full_info.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Khởi tạo Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')

In [ ]:
import ee
import geemap

# Create an interactive map
Map = geemap.Map()

# Load the DEM dataset
dataset = ee.ImageCollection('COPERNICUS/DEM/GLO30')
elevation = dataset.select('DEM')

# Define visualization parameters
elevationVis = {
    'min': 0.0,
    'max': 1000.0,
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

# Set the map center and zoom level
Map.setCenter(-6.746, 6.529, 10)

# Add the DEM layer to the map
Map.addLayer(elevation, elevationVis, 'DEM')

# Display the map
Map


Map(center=[6.529, -6.746], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
import ee
import geemap
import requests
import os

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Create the DEM image collection and select the DEM band
dataset = ee.ImageCollection('COPERNICUS/DEM/GLO30')
elevation = dataset.select('DEM').mosaic()

# Define the visualization parameters
elevationVis = {
    'min': 0.0,
    'max': 1000.0,
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

# Set the visualization parameters on the image
elevation_visualized = elevation.visualize(**elevationVis)

# Set the center coordinates and zoom level
center_lat = 22.415404
center_lon = 46.763746
zoom_level = 10

# Define the region of interest (ROI) based on the center and zoom level
scale = 100  # The scale in meters per pixel
roi = ee.Geometry.Point(center_lon, center_lat).buffer(scale * 2**zoom_level).bounds()

# Get the thumbnail URL of the image
thumbnail_params = {
    'region': roi,
    'dimensions': 512,  # you can increase this for higher resolution
    'format': 'png'
}
url = elevation_visualized.getThumbURL(thumbnail_params)

# Print the URL to download the PNG file
print("Download your PNG from this URL:", url)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path where you want to save the image
save_path = '/content/drive/MyDrive/image/elevation_map.png'

# Make sure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Download the image and save it to the specified path
response = requests.get(url)
if response.status_code == 200:
    with open(save_path, 'wb') as f:
        f.write(response.content)
    print(f"Image saved to: {save_path}")
else:
    print("Failed to download image.")


Download your PNG from this URL: https://earthengine.googleapis.com/v1/projects/ee-nguyendangkhoi9517/thumbnails/cdd1e15e61053a1397416d3af914394d-61659c6d6f7e8a75f5aa2e4c45897c78:getPixels
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Image saved to: /content/drive/MyDrive/image/elevation_map.png


In [ ]:
import ee
import geemap
import requests
import os

# Create the DEM image collection and select the DEM band
dataset = ee.ImageCollection('COPERNICUS/DEM/GLO30')
elevation = dataset.select('DEM').mosaic()

# Define the visualization parameters
elevationVis = {
    'min': 0.0,
    'max': 1000.0,
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    center_lon = row['dfo_centroid_x']
    center_lat = row['dfo_centroid_y']
    zoom_level = 10

    # Define the region of interest (ROI) based on the center and zoom level
    scale = 100  # The scale in meters per pixel
    roi = ee.Geometry.Point(center_lon, center_lat).buffer(scale * 2**zoom_level).bounds()

    # Visualize the elevation data for the given ROI
    elevation_visualized = elevation.visualize(**elevationVis)

    # Get the thumbnail URL of the image
    thumbnail_params = {
        'region': roi,
        'dimensions': 512,  # Increase for higher resolution
        'format': 'png'
    }
    url = elevation_visualized.getThumbURL(thumbnail_params)

    # Set the path where you want to save the image
    save_path = f'/content/drive/MyDrive/image/elevation_map_{index}.png'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Download the image and save it to the specified path
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Image for event {index} saved to: {save_path}")
    else:
        print(f"Failed to download image for event {index}")

Image for event 0 saved to: /content/drive/MyDrive/image/elevation_map_0.png
Image for event 1 saved to: /content/drive/MyDrive/image/elevation_map_1.png
Image for event 2 saved to: /content/drive/MyDrive/image/elevation_map_2.png
Image for event 3 saved to: /content/drive/MyDrive/image/elevation_map_3.png
Image for event 4 saved to: /content/drive/MyDrive/image/elevation_map_4.png
Image for event 5 saved to: /content/drive/MyDrive/image/elevation_map_5.png
Image for event 6 saved to: /content/drive/MyDrive/image/elevation_map_6.png
Image for event 7 saved to: /content/drive/MyDrive/image/elevation_map_7.png
Image for event 8 saved to: /content/drive/MyDrive/image/elevation_map_8.png
Image for event 9 saved to: /content/drive/MyDrive/image/elevation_map_9.png
Image for event 10 saved to: /content/drive/MyDrive/image/elevation_map_10.png
Image for event 11 saved to: /content/drive/MyDrive/image/elevation_map_11.png
Image for event 12 saved to: /content/drive/MyDrive/image/elevation_map_